In [1]:
!pip install SpeechRecognition pyttsx3 wikipedia pyjokes



In [2]:
import speech_recognition as sr
import pyttsx3
import datetime
import wikipedia
import webbrowser
import pyjokes
import re

# Initialize text-to-speech engine
engine = pyttsx3.init()

def speak(text):
    print(f"Assistant: {text}")
    engine.say(text)
    engine.runAndWait()

def wish_user():
    hour = int(datetime.datetime.now().hour)
    if hour < 12:
        speak("Good Morning!")
    elif hour < 18:
        speak("Good Afternoon!")
    else:
        speak("Good Evening!")
    speak("I am your voice assistant. Say 'start' to activate me.")

def take_command():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        recognizer.pause_threshold = 1.2
        recognizer.energy_threshold = 300
        try:
            audio = recognizer.listen(source, timeout=5)
        except sr.WaitTimeoutError:
            speak("You were silent for too long.")
            return ""

    try:
        print("Recognizing...")
        query = recognizer.recognize_google(audio, language='en-in')
        print(f"You: {query}")
        return query.lower()
    except sr.UnknownValueError:
        print("Did not understand voice input.")
        speak("Sorry, I didn't understand.")
        return ""
    except sr.RequestError as e:
        print(f"Request error: {e}")
        speak("Could not connect to the speech service.")
        return ""

def extract_topic(query):
    query = query.lower()
    query = query.replace('wikipedia', '')
    query = re.sub(r"\b(search|about|on|for|in|what is|who is|tell me)\b", '', query)
    return query.strip()

def run_assistant():
    wish_user()
    active = False
    last_question = ""

    while True:
        # 🔒 PAUSED MODE: Wait until "start" or "resume"
        while not active:
            query = take_command()
            if 'start' in query or 'resume' in query:
                active = True
                speak("I am now listening.")
            elif 'exit' in query or 'bye' in query:
                speak("Goodbye! Have a nice day!")
                return
            else:
                speak("I am paused. Say 'start' to begin or 'exit' to quit.")

        # 🔓 ACTIVE MODE: Respond to commands
        query = take_command()

        if 'stop' in query or 'pause' in query:
            active = False
            speak("I have paused. Say 'start' to continue.")
            continue

        elif 'wikipedia' in query:
            speak("Searching Wikipedia...")
            topic = extract_topic(query)
            last_question = topic
            if topic:
                try:
                    result = wikipedia.summary(topic, sentences=2)
                    speak("According to Wikipedia:")
                    speak(result)
                except wikipedia.exceptions.DisambiguationError:
                    speak("That topic is too broad. Please be more specific.")
                except wikipedia.exceptions.PageError:
                    speak("I couldn't find anything on that topic.")
                except Exception:
                    speak("Something went wrong while searching Wikipedia.")
            else:
                speak("Please say the topic you want to search on Wikipedia.")

        elif 'open youtube' in query:
            speak("Opening YouTube...")
            webbrowser.open("https://www.youtube.com")

        elif 'play' in query and 'youtube' in query:
            topic = query.replace('play', '').replace('on youtube', '').strip()
            speak(f"Playing {topic} on YouTube...")
            webbrowser.open(f"https://www.youtube.com/results?search_query={topic}")

        elif 'open pinterest' in query:
            speak("Opening Pinterest...")
            webbrowser.open("https://www.pinterest.com")

        elif 'open google' in query:
            speak("Opening Google...")
            webbrowser.open("https://www.google.com")

        elif 'time' in query:
            strTime = datetime.datetime.now().strftime("%H:%M:%S")
            speak(f"The current time is {strTime}")

        elif 'joke' in query:
            joke = pyjokes.get_joke()
            speak(joke)

        elif 'your name' in query:
            speak("I am your personal voice assistant.")
        elif 'who made you' in query or 'who created you' in query:
            speak("I was created by you using Python.")
        elif 'tell me about yourself' in query:
            speak("I am a Python-based voice assistant designed to help you with tasks like search, time, jokes, and more.")

        elif 'tell me more' in query and last_question:
            speak("Let me expand on that...")
            try:
                result = wikipedia.summary(last_question, sentences=5)
                speak(result)
            except:
                speak("I couldn’t find more information.")

        elif 'exit' in query or 'bye' in query:
            speak("Goodbye! Have a nice day!")
            break

        else:
            speak("I didn't understand that. Should I search it on Google?")
            confirmation = take_command()
            if 'yes' in confirmation or 'okay' in confirmation:
                speak("Searching on Google...")
                webbrowser.open(f"https://www.google.com/search?q={query}")
            else:
                speak("Okay, let’s try something else.")



In [4]:
run_assistant()

Assistant: Good Evening!
Assistant: I am your voice assistant. Say 'start' to activate me.
Listening...
Recognizing...
You: start
Assistant: I am now listening.
Listening...
Recognizing...
You: open Google
Assistant: Opening Google...
Listening...
Recognizing...
Did not understand voice input.
Assistant: Sorry, I didn't understand.
Assistant: I didn't understand that. Should I search it on Google?
Listening...
Recognizing...
Did not understand voice input.
Assistant: Sorry, I didn't understand.
Assistant: Okay, let’s try something else.
Listening...
Recognizing...
You: exit
Assistant: Goodbye! Have a nice day!
